In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import glob as glob
import rdkit.Chem
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split


# Predicting boiling points based on generic molecule information.
* To test and train a model a data set is mandatory. We will stick to the set accesible on the  
    quantitative structure−activity relationships database (qsarDB): https://qsardb.org/repository/handle/10967/158  

* It provides us with boiling points and some other information on the molecule like the number of specific atoms or functional groups and connectivity index.

* You can find the boiling points in the file '1999JCIM491/properties/Tb/values' (seperated by tabs)

* The for each descriptor is a subfolder in '1999JCIM491/descriptors' where you will also find a 'values' file
    * glob.glob('to/dir/*/') provides a list of all subfolders in folder 'to/dir/'


In [ ]:
##Import the target and descriptors (features)
descriptor_folder = '1999JCIM491/descriptors/'
property_file = '1999JCIM491/properties/Tb/values'

properties = pd.DataFrame()#import boiling temperature
descriptor = pd.DataFrame()#import descriptors

# Feature processing

* Typically the features need to be standardized and normalized to achieve a functioning basis for an ML model.

* Luckily decision trees do not depend on these, and therefore we do not have to take care of this. 

* Just keep in mind, that if you use other algorithms the feature processing might be essential.

# Test and Training

* For validation the data set needs to be divided into a test and training set. 

* Typically, 75 \% of the data set is used for training, while the other 25 % are used for testing the model.

* The model can than be fitted with the training set.

In [ ]:


properties_train,properties_test,descriptors_train,descriptors_test = train_test_split(properties,descriptors,test_size=0.25,random_state=42)

Feature_train = descriptors_train

Target_train = properties_train

MLmodel = DecisionTreeRegressor(max_depth=2)

MLmodel.fit(Feature_train,Target_train)

# Visualization and Analysis

* There are several tools and techniques to analyze the quality of an ML model. In the following are some examples

* Specific for decision trees is the visualization of them by plotting the tree with an in-build function.  
However, this only makes sense for low tree depth.

* This makes them easy to interpret and easy to follow the ongoing routines inside the prediction.

In [ ]:
from sklearn import tree
tree.plot_tree(MLmodel)

## Prediction 
* The test set can be now easily predicted as shown here

In [ ]:
Feature_test = descriptors_test
Target_test = properties_test

Pred_Target = MLmodel.predict(Feature_test)

## Box plots
* Furthermore, the tested data can be predicted and compared in a box plot, where the true property is plotted against the prediction.

In [ ]:
## Plot the prediction and the true values of the boiling points.

* As you can see from this example, the depth of the model is chosen too low and therefore only 4 different values are predicted.

* Redo the fit without specifying the maximum depth. In such case, the depth of the tree increases until all leaves are pure.

## Learning plots

* Examining the learning procedure of a model is basic analysis to see how significant the model benefits from an increase of data.

* In a learning plot a measurement of the accuracy of a model is plotted against the amount of data used for the fit.

    * The measurement of accuracy can be for example the RMSE or MAE 

    * To achieve a statistical more significant result do the fit for every training set size for several random seeds

    * If the model actually learns from the provided features the accuracy should increase with increasing training set size

# Model Optimization

* ML algorithms rely on several predefined parameters, which all can be optimized.

* There are automatized in-build functions in sklearn learn, which allow a grid based search for the best performing parameter ensemble.

* Either the complete parameter grid can be screened or a predefined number of random parameter ensembles is tested. 

* For this the algorithm is wrapped in a search function and a grid or distribution of the parameters has to be provided.

### Some info to the given parameters

* 'max_depth' gives the maximum depth of the tree

* 'ccp_alpha' is a parameter that describes how much the tree is pruned. Pruning is used, as the tree can tend to overfit. Pruning reduces the number of leaves in the tree.

* 'max_features' is the number of features when looking at the best split.

### Parallelization 

* As these optimization procedures can get quite costly, it is helpful to use parallelization

* This is conveniently implemented in sklearn and all we have to do is to provide a number of jobs.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import numpy as np

params = {'max_depth' : [2,5,10,1000000000],
          'ccp_alpha' : [False,0.001,0.002,0.005],
          'max_features' : [0.5,0.75,1.0]}

MLmodel = GridSearchCV(DecisionTreeRegressor(),param_grid=params,n_jobs=8)

MLmodel.fit(Feature_train,Target_train)

MLmodel.best_params_

* Another possibility for the optimization of the model is to use a more proficient algorithm.

*  For example ensemble based methods can be used. These rely on the several decision trees from which an average is constructed for the prediction.

* Also for this method a parameter optimization can be useful. Another important quantity is the number of estimators (so of the single decision trees) should be included for the parameter optimization.

* Redo the parameter search but with the ensemble method and fit the test system again.

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

params = {'max_depth' : [2,5,10,100,10000],
          'ccp_alpha' : [False,0.001,0.002,0.005],
          'max_features' : [0.1,0.25,0.5,0.75,1.0],
          'n_estimators' : [50,100,300,500]}

MLmodel = GridSearchCV(ExtraTreesRegressor(),param_grid=params)

MLmodel.set_params(n_jobs=8)
MLmodel.fit(Feature_train,Target_train)
MLmodel.best_params_

## Feature Reduction

* Tree based algorithms can also be enhanced by a feature reduction, if the chosen features only have a low variance throughout the data set.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.0)
selected_Feature = selector.fit(Features)

## New Features

* However, there might be also other features which have a significant influence on the performance.

* One could for example also include the masses of the molecule.

* The package rdkit provides also functions which return number of specific fragments. 

* You can access the structure of the molecule via the InChI of each system.

In [ ]:
from rdkit.Chem import FunctionalGroups,Descriptors,Fragments

inchi_file = '1999JCIM491/compounds/compounds.xml'

inchis = pd.read_xml(inchi_file,parser='lxml')

properties = pd.read_csv(property_file,index_col=0,sep='\t')

wt = []
HA_wt = []
mols = []
hierachy = []

for i in range(len(inchis['InChI'])):
    a = rdkit.Chem.inchi.MolFromInchi(inchis['InChI'][i],removeHs=True)
    mols.append(a)
    hierachy.append(i)
    wt.append(Descriptors.ExactMolWt(a))
    HA_wt.append(Descriptors.HeavyAtomMolWt(a))


desc = Fragments.fr_Al_OH(a)

descriptors['wt'] = wt
descriptors['HA_wt'] = HA_wt
